## Testing DFA learning with a string rewriting system

In [10]:
import copy, sys, pandas as pd
import random 

sys.path.append("../")
sys.path.append("../inferring")
sys.path.append("../utils/DFA")
from importlib import reload


import inferring.Inferring as Inferring, inferring.InferringDFA as InferringDFA
import utils.automats.DFA
import utils.advice_systems.SRS as SRS

reload(SRS)
reload(Inferring)
reload(InferringDFA)
reload(utils.automats.DFA)

from inferring.Inferring import Inferring
from utils.automats.DFA.DFA import DFA
from utils.advice_systems.SRS import SRS 

from inferring.InferringDFA import InferringDFA

In [11]:
def run_learning_process(test, advice_system=None, check_consistency=False, equiv_query_fashion="BFS", debug=False):
    _dfa = copy.deepcopy(test.dfa)
    learn_dfa = InferringDFA(_dfa, 
                                advice_system, 
                                check_consistency=check_consistency, 
                                equiv_query_fashion=equiv_query_fashion,
                                debug=debug)
        
    dfa, cnt, cnt_ex = learn_dfa.run(counterexamples=True)
    return copy.deepcopy(dfa), cnt, len(cnt_ex) +1 

In [12]:
class test_example():
    def __init__(self, dfa, descrip=''):
        self.dfa = copy.deepcopy(dfa)
        self.descrip = descrip

In [13]:
def create_table(columns, data):
    columns = pd.MultiIndex.from_tuples(columns)
    df = pd.DataFrame(data, columns=columns)

    avg_mq_red = df[(('Reduction', 'MQ'))].mean()
    avg_eq_red = df[(('Reduction', 'EQ'))].mean()

    max_mq_red = df[(('Reduction', 'MQ'))].max()
    max_eq_red = df[(('Reduction', 'EQ'))].max()

    min_mq_red = df[(('Reduction', 'MQ'))].min()
    min_eq_red = df[(('Reduction', 'EQ'))].min()

    empty_row = pd.DataFrame([[""] * df.shape[1]], columns=df.columns)

    def set_params(row, params):
        for k, v in params:
            row[columns[k]] = v 

    max_red_row = copy.deepcopy(empty_row)
    set_params(row=max_red_row, params=[(0, "Max reduction"), (-2, int(max_mq_red)), (-1, int(max_eq_red))])
    min_red_row = copy.deepcopy(empty_row)
    set_params(row=min_red_row, params=[(0, "Min reduction"), (-2, int(min_mq_red)), (-1, int(min_eq_red))])
    avg_red_row = copy.deepcopy(empty_row)
    set_params(row=avg_red_row, params=[(0, "Average reduction"), (-2, int(avg_mq_red)), (-1, int(avg_eq_red))])

    rows = [empty_row, max_red_row, min_red_row, avg_red_row]
    for row in rows:
        df = pd.concat([df, row], ignore_index=True)

    df[('Reduction', 'MQ')] = df[('Reduction', 'MQ')].apply(lambda x: f'{x}%' if isinstance(x, int) else x)
    df[('Reduction', 'EQ')] = df[('Reduction', 'EQ')].apply(lambda x: f'{x}%' if isinstance(x, int) else x)
    return df    

### Concurrent systems

#### Convolution of patterns-dfa (dfa recognizing patterns)

In [14]:
def random_patterns(input_signs, n=5, max_length=10):
    patterns = []
    for _ in range(n):
        p = ''.join(random.choices(population=input_signs,k=random.randint(max_length//2, max_length)))
        while p in set(patterns):
            p = ''.join(random.choices(population=input_signs,k=random.randint(max_length//2, max_length)))
        patterns.append(p)
    return patterns

In [15]:
class RecordPattern:
    def __init__(self, d1, d1_type, d2, d2_type, conv, cnt_normal, cnt_with_advice):
        pdfa_types = {DFA.OR_TYPE_PATTERN_DFA: "OR", DFA.AND_TYPE_PATTERN_DFA:"AND"}
        
        self.d1 = d1
        self.d2 = d2
        self.d1_type = pdfa_types[d1_type]
        self.d2_type = pdfa_types[d2_type] 
        
        self.conv = conv
        self.cnt_normal = cnt_normal
        self.cnt_with_advice = cnt_with_advice

        self.mq_reduction = int(((cnt_normal[0] - cnt_with_advice[0][0])/cnt_normal[0])*100)
        self.eq_reduction = int(((cnt_normal[1] - cnt_with_advice[0][1])/cnt_normal[1])*100)

    def print_record(self):
        print(f"|d1| = {self.d1}, |d2| = {self.d2}, |conv| = {self.conv}, (mq, eq) = {self.cnt_normal} vs (mq, eq) = {self.cnt_with_advice}, mq_reduction = {self.mq_reduction}, eq_reduction ={self.eq_reduction}") 

In [ ]:
number_of_itreations = 25
seeds =  [i for i in range(number_of_itreations)]
input_signs = ['a', 'b', 'c', 'd']

number_of_patterns = 2
max_pattern_length = 10

results = []

i = 0 
while i < number_of_itreations:
    random.seed(seeds[i])
    i += 1 
    print(f"iter nr: {i}")

    dfa1, dfa2 = DFA(), DFA()
    dfa1.create_pattern_dfa(input_signs=input_signs, patterns=random_patterns(input_signs, number_of_patterns, max_pattern_length), _type=DFA.OR_TYPE_PATTERN_DFA if i&1 else DFA.AND_TYPE_PATTERN_DFA)
    dfa2.create_pattern_dfa(input_signs=input_signs, patterns=random_patterns(input_signs, number_of_patterns, max_pattern_length), _type=DFA.OR_TYPE_PATTERN_DFA if i&1 else DFA.AND_TYPE_PATTERN_DFA)

    d1, _, _ = run_learning_process(test_example(dfa=copy.deepcopy(dfa1)))
    d2, _, _ = run_learning_process(test_example(dfa=copy.deepcopy(dfa2)))

    conv_dfa = DFA() 
    conv_dfa.create_convolution(dfa1=copy.deepcopy(d1), dfa2=copy.deepcopy(d2))

    d, cnt_with_advice, cnt_ex_with_advice = run_learning_process(test=test_example(dfa=conv_dfa), advice_system=SRS(), check_consistency=True) 
    d.type = DFA.CONV_DFA
    if d.Q > 1000:
        continue
    _, cnt_normal, _ = run_learning_process(test=test_example(dfa=d), advice_system=None, check_consistency=False)

    results.append(RecordPattern(d1=d1.Q, d1_type = dfa1.type, d2=d2.Q, d2_type=dfa2.type, conv=d.Q, cnt_normal=cnt_normal, cnt_with_advice=(cnt_with_advice, cnt_ex_with_advice)))

In [19]:
results.sort(key=lambda x: -100000 +x.conv if x.d1_type=="OR" else x.conv)

In [20]:
columns = [('Target language', 'conv(DFA1, DFA2)'), ('Target language', 'DFA1'), ('Target language', 'DFA2'), ('L* without advice', 'MQ'), ('L* without advice', 'EQ'), ('L* with advice', 'MQ'), ('L* with advice', 'EQ'), ('Reduction', 'MQ'), ('Reduction', 'EQ')]
data = {
    columns[0]: [r.conv for r in results],
    columns[1]: [(r.d1, r.d1_type) for r in results],
    columns[2]: [(r.d2, r.d2_type) for r in results],
    columns[3]: [r.cnt_normal[0] for r in results],
    columns[4]: [r.cnt_normal[1] for r in results],
    columns[5]: [r.cnt_with_advice[0][0] for r in results],
    columns[6]: [(r.cnt_with_advice[0][1], r.cnt_with_advice[1]) for r in results],
    columns[7]: [r.mq_reduction for r in results],
    columns[8]: [r.eq_reduction for r in results]
}

df = create_table(columns=columns, data=data)
display(df)

Target language                       L* without advice       \
     conv(DFA1, DFA2)       DFA1       DFA2                MQ   EQ   
0                 132   (11, OR)   (12, OR)             91707   49   
1                 169   (13, OR)   (13, OR)            170665   64   
2                 169   (13, OR)   (13, OR)            142249   58   
3                 180   (15, OR)   (12, OR)            118714   49   
4                 182   (13, OR)   (14, OR)            223307   70   
5                 195   (13, OR)   (15, OR)            183239   64   
6                 208   (16, OR)   (13, OR)            226043   84   
7                 216   (12, OR)   (18, OR)            263478   69   
8                 224   (16, OR)   (14, OR)            202625   67   
9                 225   (15, OR)   (15, OR)            305969   70   
10                289   (17, OR)   (17, OR)            540697  108   
11                494  (26, AND)  (19, AND)           1013981  124   
12                612  (34, AND)  (18, AND)           3698567  223   
13                720  (30, AND)  (24, AND)           2274211  199   
14                816  (34, AND)  (24, AND)           3205809  244   
15                924  (28, AND)  (33, AND)           3668847  268   
16                952  (28, AND)  (34, AND)           4373192  220   
17                                                                   
18      Max reduction                                                
19      Min reduction                                                
20  Average reduction                                                

   L* with advice           Reduction       
               MQ        EQ        MQ   EQ  
0           44664   (4, 49)       51%  91%  
1           71033   (4, 64)       58%  93%  
2           67078   (4, 58)       52%  93%  
3           62649   (4, 49)       47%  91%  
4           77653   (4, 70)       65%  94%  
5           78666   (4, 64)       57%  93%  
6          106100   (4, 84)       53%  95%  
7           97009   (4, 69)       63%  94%  
8          102337   (4, 67)       49%  94%  
9           97880   (4, 70)       68%  94%  
10         177628  (4, 108)       67%  96%  
11         380094  (4, 124)       62%  96%  
12         616675  (4, 174)       83%  98%  
13         830729  (4, 214)       63%  97%  
14        1037829  (4, 244)       67%  98%  
15        1306835  (4, 268)       64%  98%  
16        1162988  (4, 220)       73%  98%  
17                                          
18                                83%  98%  
19                                47%  91%  
20                                61%  94%

#### Convolution of 2 random DFA 

In [101]:
class Record:
    def __init__(self, d1, d2, conv, cnt_normal, cnt_with_advice):
        self.d1 = d1
        self.d2 = d2
        self.conv = conv
        self.cnt_normal = cnt_normal
        self.cnt_with_advice = cnt_with_advice

        self.mq_reduction = int(((cnt_normal[0] - cnt_with_advice[0][0])/cnt_normal[0])*100)
        self.eq_reduction = int(((cnt_normal[1] - cnt_with_advice[0][1])/cnt_normal[1])*100)
    def print_record(self):
        print(f"|d1| = {self.d1}, |d2| = {self.d2}, |conv| = {self.conv}, (mq, eq) = {self.cnt_normal} vs (mq, eq) = {self.cnt_with_advice}, mq_reduction = {self.mq_reduction}, eq_reduction ={self.eq_reduction}") 

In [ ]:
number_of_itreations = 5
seeds =  [i for i in range(number_of_itreations)]
input_signs = ['a', 'b', 'c', 'd']
max_number_of_states = 30

results = []

i = 0 
while i < number_of_itreations: 
    random.seed(seeds[i])
    i += 1
    print(f"iter nr: {i}")

    dfa1 = DFA()
    dfa2 = DFA()

    dfa1.create_random_dfa(Q=random.randint(max_number_of_states//2,max_number_of_states), input_signs=input_signs)
    dfa2.create_random_dfa(Q=random.randint(max_number_of_states//2,max_number_of_states), input_signs=input_signs)
    
    d1, _, _ = run_learning_process(test_example(dfa=copy.deepcopy(dfa1)))   
    d2, _, _ = run_learning_process(test_example(dfa=copy.deepcopy(dfa2)))
    conv_dfa = DFA()
    conv_dfa.create_convolution(dfa1=d1, dfa2=d2) 
    # print(f"|d1| = {d1.Q}, |F1| = {len(d1.F)}; |d2| = {d2.Q}, |F2| = {len(d2.F)} ")
    # print(f"rozmiar conwolucji: {conv_dfa.Q}")
    
    d, cnt_with_advice, cnt_ex_with_advice = run_learning_process(test=test_example(dfa=conv_dfa), advice_system=SRS(), check_consistency=True) 
    d.type = DFA.CONV_DFA
    _, cnt_normal, _ = run_learning_process(test=test_example(dfa=d), advice_system=None, check_consistency=False)

    results.append(Record(d1=d1.Q, d2=d2.Q, conv=d.Q, cnt_normal=cnt_normal, cnt_with_advice=(cnt_with_advice, cnt_ex_with_advice)))

In [114]:
results.sort(key=lambda x: x.conv)

In [115]:
columns = [('Target language', 'conv(DFA1, DFA2)'), ('Target language', 'DFA1'), ('Target language', 'DFA2'), ('L* without advice', 'MQ'), ('L* without advice', 'EQ'), ('L* with advice', 'MQ'), ('L* with advice', 'EQ'), ('Reduction', 'MQ'), ('Reduction', 'EQ')]
data = {
    columns[0]: [r.conv for r in results],
    columns[1]: [r.d1 for r in results],
    columns[2]: [r.d2 for r in results],
    columns[3]: [r.cnt_normal[0] for r in results],
    columns[4]: [r.cnt_normal[1] for r in results],
    columns[5]: [r.cnt_with_advice[0][0] for r in results],
    columns[6]: [(r.cnt_with_advice[0][1], r.cnt_with_advice[1]) for r in results],
    columns[7]: [r.mq_reduction for r in results],
    columns[8]: [r.eq_reduction for r in results]
}

df = create_table(columns=columns, data=data)
display(df)

Target language           L* without advice      L* with advice  \
    conv(DFA1, DFA2) DFA1 DFA2                MQ   EQ             MQ   
0                210   14   15            303236  133         114941   
1                270   18   15            325522  110         148804   
2                352   22   16            801477  165         258904   
3                550   22   25            659071  124         245866   
4                676   26   26           1543634  168         591684   
5                                                                      
6      Max reduction                                                   
7      Min reduction                                                   
8  Average reduction                                                   

            Reduction       
         EQ        MQ   EQ  
0  (2, 106)       62%  98%  
1  (2, 104)       54%  98%  
2  (3, 141)       67%  98%  
3   (2, 77)       62%  98%  
4  (2, 166)       61%  98%  
5                           
6                 67%  98%  
7                 54%  98%  
8                 61%  98%

### Synchronizing words

In [7]:
class RecordSYNCH:
    def __init__(self, d, reset_word, cnt_normal, cnt_with_advice):
        self.d = d
        self.reset_word = reset_word
        self.cnt_normal = cnt_normal
        self.cnt_with_advice = cnt_with_advice

        self.mq_reduction = int(((cnt_normal[0] - cnt_with_advice[0][0])/cnt_normal[0])*100)
        self.eq_reduction = int(((cnt_normal[1] - cnt_with_advice[0][1])/cnt_normal[1])*100)
    def print_record(self):
        print(f"|d1| = {self.d}, |reset_word| = {len(self.reset_word)}, cnt_normal = {self.cnt_normal} vs cnt_with_advice = {self.cnt_with_advice}")

In [9]:
number_of_itreations = 10
seeds = [i for i in range(number_of_itreations)]

input_signs = ['a', 'b', 'c', 'd']
max_number_of_states = 20

results = []

good_dfs = 0

i = 0 
while i < number_of_itreations: 
    print(f"iter = {i+1}")
    random.seed(seeds[i])
    i += 1

    reset_word = DFA.NOT_RESETING_WORD
    while True:

        dfa_ = DFA()
        dfa_.create_random_dfa(Q=random.randint(max_number_of_states//2,max_number_of_states), input_signs=input_signs)
        d, cnt_normal, _ = run_learning_process(test_example(dfa=copy.deepcopy(dfa_)))   

        print(f"d = {d.Q}")
        reset_word_zyzik = d.find_minim_synch_word()
        ans, reset_word = d.check_synchronicity()
        print(f"reset_word = {reset_word}, reset_word_zyzik = {reset_word_zyzik}")
        if reset_word!=DFA.NOT_RESETING_WORD:
            d.type = DFA.SYNCHRONICITY
            d.reset_word = reset_word
            good_dfs += 1 
            break
    print("\n")
    
    d_with_advice, cnt_with_advice, cnt_ex_with_advice = run_learning_process(test=test_example(dfa=d), advice_system=SRS(), check_consistency=True, debug=False) 
    results.append(RecordSYNCH(d=d.Q, reset_word=reset_word, cnt_normal=cnt_normal, cnt_with_advice=(cnt_with_advice, cnt_ex_with_advice)))

print(f"good dfs = {good_dfs}")

iter = 1
d = 16
reset_word = acaccbdaadd, reset_word_zyzik = -1


iter = 2
d = 1
reset_word = , reset_word_zyzik = -1


iter = 3
d = 1
reset_word = , reset_word_zyzik = -1


iter = 4
d = 13
reset_word = bddda, reset_word_zyzik = -1


iter = 5
d = 12
reset_word = adbad, reset_word_zyzik = -1


iter = 6
d = 19
reset_word = abaabababd, reset_word_zyzik = -1


iter = 7
d = 19
reset_word = adadaccdcaaddadd, reset_word_zyzik = -1


iter = 8
d = 1
reset_word = , reset_word_zyzik = -1


iter = 9
d = 13
reset_word = cacddcb, reset_word_zyzik = -1


iter = 10
d = 17
reset_word = cbcbbbbdabbba, reset_word_zyzik = -1


good dfs = 10


In [147]:
results.sort(key=lambda x: x.d)

In [ ]:
columns = [('', 'DFA'), ('L* without advice', 'MQ'), ('L* without advice', 'EQ'), ('L* with advice', 'MQ'), ('L* with advice', 'EQ'), ('Reduction', 'MQ'), ('Reduction', 'EQ')]
data = {
    columns[0]: [r.d for r in results],
    columns[1]: [r.cnt_normal[0] for r in results],
    columns[2]: [r.cnt_normal[1] for r in results],
    columns[3]: [r.cnt_with_advice[0][0] for r in results],
    columns[4]: [(r.cnt_with_advice[0][1], r.cnt_with_advice[1]) for r in results],
    columns[5]: [r.mq_reduction for r in results],
    columns[6]: [r.eq_reduction for r in results]
}

df = create_table(columns=columns, data=data)
display(df)

### Marked words

In [7]:
class RecordMARKED:
    def __init__(self, d, cnt_normal, cnt_with_advice):
        self.d = d
        self.cnt_normal = cnt_normal
        self.cnt_with_advice = cnt_with_advice

        self.mq_reduction = int(((cnt_normal[0] - cnt_with_advice[0][0])/cnt_normal[0])*100)
        self.eq_reduction = int(((cnt_normal[1] - cnt_with_advice[0][1])/cnt_normal[1])*100)
    def print_record(self):
        print(f"|d| = {self.d}, |cnt_normal| = {self.cnt_normal} vs |cnt_with_advice| = {self.cnt_with_advice}") 

In [9]:
number_of_itreations = 10
seeds = [i+1 for i in range(number_of_itreations)]
input_signs = ['a', 'b', 'c', 'd'] 
max_number_of_states = 500

results = []

i = 0 
while i < number_of_itreations:  
    random.seed(seeds[i])
    i += 1

    dfa_ = DFA()
    dfa_.create_random_dfa(Q=random.randint(max_number_of_states//2,max_number_of_states), input_signs=input_signs)
    d, cnt_normal, _ = run_learning_process(test_example(dfa=copy.deepcopy(dfa_)), advice_system=None, check_consistency=False)   
    d_marked = d.create_marked_words_atomaton()

    # _, cnt_normal, _ = run_learning_process(test=test_example(dfa=d_marked))
    _, cnt_with_advice, cnt_ex_with_advice = run_learning_process(test=test_example(dfa=d_marked), advice_system=SRS(), check_consistency=True) 

    results.append(RecordMARKED(d=d.Q, cnt_normal=cnt_normal, cnt_with_advice=(cnt_with_advice, cnt_ex_with_advice)))

In [10]:
results.sort(key=lambda x: x.d)

In [11]:
columns = [('', 'DFA'), ('L* without advice', 'MQ'), ('L* without advice', 'EQ'), ('L* with advice', 'MQ'), ('L* with advice', 'EQ'), ('Reduction', 'MQ'), ('Reduction', 'EQ')]
data = {
    columns[0]: [r.d for r in results],
    columns[1]: [r.cnt_normal[0] for r in results],
    columns[2]: [r.cnt_normal[1] for r in results],
    columns[3]: [r.cnt_with_advice[0][0] for r in results],
    columns[4]: [(r.cnt_with_advice[0][1], r.cnt_with_advice[1]) for r in results],
    columns[5]: [r.mq_reduction for r in results],
    columns[6]: [r.eq_reduction for r in results]
}

df = create_table(columns=columns, data=data)
display(df)

L* without advice     L* with advice            \
                  DFA                MQ  EQ             MQ        EQ   
0                 277             35221  36          39993  (39, 40)   
1                 301             48213  44          58001  (45, 46)   
2                 305             62593  57          73533  (52, 53)   
3                 306             52689  43          61734  (49, 50)   
4                 324             47028  34          50634  (37, 38)   
5                 358             58408  43          62396  (43, 44)   
6                 383             55583  39          60936  (39, 40)   
7                 402             53510  36          65152  (39, 40)   
8                 447             55469  32          64303  (38, 39)   
9                 487             85283  43          81775  (45, 46)   
10                                                                     
11      Max reduction                                                  
12      Min reduction                                                  
13  Average reduction                                                  

   Reduction        
          MQ    EQ  
0       -13%   -8%  
1       -20%   -2%  
2       -17%    8%  
3       -17%  -13%  
4        -7%   -8%  
5        -6%    0%  
6        -9%    0%  
7       -21%   -8%  
8       -15%  -18%  
9         4%   -4%  
10                  
11        4%    8%  
12      -21%  -18%  
13      -12%   -5%

### DFA with partial specification

In [19]:
class RecordPARTIAL:
    def __init__(self, d, cnt_normal, cnt_with_advice, srs_size):
        self.d = d
        self.cnt_normal = cnt_normal
        self.cnt_with_advice = cnt_with_advice
        self.srs_size = srs_size

        self.mq_reduction = int(((cnt_normal[0] - cnt_with_advice[0][0])/cnt_normal[0])*100)
        self.eq_reduction = int(((cnt_normal[1] - cnt_with_advice[0][1])/cnt_normal[1])*100)
        
    def print_record(self):
        print(f"|d1| = {self.d}, |d_cnt_normal| = {self.cnt_normal} vs |d_cnt_with_partial_advice| = {self.cnt_with_partial_advice}") 

In [ ]:
number_of_itreations = 100
seeds = [i+1 for i in range(number_of_itreations)]
input_signs = ['a', 'b', 'c', 'd']
max_number_of_states = 1000 

results = []

i = 0 
while i < number_of_itreations:  
    random.seed(seeds[i])
    i += 1
    print(f"iter{i}")

    dfa_ = DFA()
    dfa_.create_random_dfa(Q=random.randint(max_number_of_states//3,max_number_of_states), input_signs=input_signs)
    dfa_partial = copy.deepcopy(dfa_)
    dfa_partial.prune()

    dfa_to_learn = dfa_.create_coppy_with_start_sign()

    d, cnt_normal, _ = run_learning_process(test_example(dfa=copy.deepcopy(dfa_to_learn)), advice_system=None, check_consistency=False)   
    _, cnt_with_advice, cnt_ex_with_advice, srs_size = run_learning_process(test=test_example(dfa=dfa_to_learn), advice_system=dfa_partial, check_consistency=True, return_srs_size=True) 

    results.append(RecordPARTIAL(d=d.Q, cnt_normal=cnt_normal, cnt_with_advice=(cnt_with_advice, cnt_ex_with_advice), srs_size=srs_size))

In [21]:
results.sort(key = lambda x: x.d)

In [22]:
columns = [('', 'DFA'), ('', 'SRS_size'), ('L* without advice', 'MQ'), ('L* without advice', 'EQ'), ('L* with advice', 'MQ'), ('L* with advice', 'EQ'), ('Reduction', 'MQ'), ('Reduction', 'EQ')]
data = {
    columns[0]: [r.d for r in results],
    columns[1]: [r.srs_size for r in results],
    columns[2]: [r.cnt_normal[0] for r in results],
    columns[3]: [r.cnt_normal[1] for r in results],
    columns[4]: [r.cnt_with_advice[0][0] for r in results],
    columns[5]: [(r.cnt_with_advice[0][1], r.cnt_with_advice[1]) for r in results],
    columns[6]: [r.mq_reduction for r in results],
    columns[7]: [r.eq_reduction for r in results]
}

df = create_table(columns=columns, data=data)
display(df)

L* without advice     L* with advice            \
                  DFA SRS_size                MQ  EQ             MQ        EQ   
0                 374        9             82709  36          59948  (35, 36)   
1                 387        9             85582  40          62028  (39, 40)   
2                 434        9             90758  37          65636  (36, 37)   
3                 462        9            116949  50          85132  (49, 50)   
4                 469        9            116843  44          85011  (43, 44)   
5                 473        9            100802  39          72946  (38, 39)   
6                 495        9            101526  39          73360  (38, 39)   
7                 512        9             96815  34          69724  (33, 34)   
8                 539        9            125645  39          91205  (38, 39)   
9                 548        9            114584  38          82850  (37, 38)   
10                565        9            115876  36          83720  (35, 36)   
11                566        9            120611  40          87268  (39, 40)   
12                583        9            124232  34          89886  (33, 34)   
13                615        9            138431  41         100355  (40, 41)   
14                654        9            173376  46         126352  (45, 46)   
15                694        9            186753  49         136156  (48, 49)   
16                716        9            178346  51         129718  (50, 51)   
17                783        9            220093  53         160653  (52, 53)   
18                792        9            197270  42         145851  (42, 42)   
19                812        9            247736  51         181226  (50, 51)   
20                858        9            227436  55         165724  (54, 55)   
21                901        9            206386  40         149678  (39, 40)   
22                901        9            202781  45         146973  (44, 45)   
23                955        9            333382  64         244652  (63, 64)   
24                                                                              
25      Max reduction                                                           
26      Min reduction                                                           
27  Average reduction                                                           

   Reduction      
          MQ  EQ  
0        27%  2%  
1        27%  2%  
2        27%  2%  
3        27%  2%  
4        27%  2%  
5        27%  2%  
6        27%  2%  
7        27%  2%  
8        27%  2%  
9        27%  2%  
10       27%  2%  
11       27%  2%  
12       27%  2%  
13       27%  2%  
14       27%  2%  
15       27%  2%  
16       27%  1%  
17       27%  1%  
18       26%  0%  
19       26%  1%  
20       27%  1%  
21       27%  2%  
22       27%  2%  
23       26%  1%  
24                
25       27%  2%  
26       26%  0%  
27       26%  1%

### Idempotent actions

In [29]:
class RecordIDEM:
    def __init__(self, d, cnt_normal, cnt_with_advice):
        self.d = d
        self.cnt_normal = cnt_normal
        self.cnt_with_advice = cnt_with_advice

        self.mq_reduction = int(((cnt_normal[0] - cnt_with_advice[0][0])/cnt_normal[0])*100)
        self.eq_reduction = int(((cnt_normal[1] - cnt_with_advice[0][1])/cnt_normal[1])*100)
    def print_record(self):
        print(f"|d| = {self.d}, |cnt_normal| = {self.cnt_normal} vs |cnt_with_advice| = {self.cnt_with_advice}") 

In [ ]:
number_of_itreations = 10
seeds = [i for i in range(number_of_itreations)]
input_signs = ['a', 'b', 'c', 'd']
max_number_of_states =1000 

results = []

i = 0 
while i < number_of_itreations:  
    random.seed(seeds[i])
    i += 1
    print(f"iter: {i}")

    dfa_ = DFA()

    dfa_.create_random_indempotent_automaton(Q=random.randint(max_number_of_states//2,max_number_of_states), input_signs=input_signs)
    d, cnt_normal, _ = run_learning_process(test=test_example(dfa=dfa_))
    d.type = DFA.INDEMPOTENT
    while not d.check_if_idempotent():
        dfa_.create_random_indempotent_automaton(Q=random.randint(max_number_of_states//2,max_number_of_states), input_signs=input_signs)
        d, cnt_normal, _ = run_learning_process(test=test_example(dfa=dfa_))
        d.type = DFA.INDEMPOTENT

    _, cnt_with_advice, cnt_ex_with_advice = run_learning_process(test=test_example(dfa=d), advice_system=SRS(), check_consistency=True, debug=False) 

    results.append(RecordIDEM(d=d.Q, cnt_normal=cnt_normal, cnt_with_advice=(cnt_with_advice, cnt_ex_with_advice)))

In [31]:
results.sort(key=lambda x: x.d)

In [32]:
columns = [('', 'DFA'), ('L* without advice', 'MQ'), ('L* without advice', 'EQ'), ('L* with advice', 'MQ'), ('L* with advice', 'EQ'), ('Reduction', 'MQ'), ('Reduction', 'EQ')]
data = {
    columns[0]: [r.d for r in results],
    columns[1]: [r.cnt_normal[0] for r in results],
    columns[2]: [r.cnt_normal[1] for r in results],
    columns[3]: [r.cnt_with_advice[0][0] for r in results],
    columns[4]: [(r.cnt_with_advice[0][1], r.cnt_with_advice[1]) for r in results],
    columns[5]: [r.mq_reduction for r in results],
    columns[6]: [r.eq_reduction for r in results]
}

df = create_table(columns=columns, data=data)
display(df)

L* without advice     L* with advice           Reduction  \
                 DFA                MQ  EQ             MQ        EQ        MQ   
0                561             84761  33          71756  (24, 34)       15%   
1                603             58523  22          63005  (18, 26)       -7%   
2                604             98506  42          96947  (31, 44)        1%   
3                962            165521  41         166803  (35, 48)        0%   
4                                                                               
5      Max reduction                                                      15%   
6      Min reduction                                                      -7%   
7  Average reduction                                                       2%   

        
    EQ  
0  27%  
1  18%  
2  26%  
3  14%  
4       
5  27%  
6  14%  
7  21%

<!-- ### Uczenie się splotów - **debugowanie**  -->